# LMA: Levenberg-Marquardt Algorithm

The Levenberg-Marquardt algorithm is an iterative procedure widely used for solving non-linear least squares problems or for finding roots of non-linear systems of equations. This implementation is designed to be robust and offer maximum flexibility, but at the same time provides sensible defaults to facilitate its usage.

## Introduction

The LMA method interpolates between the more aggressive Gauss-Newton algorithm and the more conservative method of gradient descent to find the set of *solution parameters* $p$ that minimize a cost function $F(p)$, defined as:

$$F(p) = \sum_i \rho(y_i(p), c_i)$$

where $y_i(p)$ are the components of the residuals vector $y$, and $\rho$ is a loss function which depends on the residuals and a scaling factor $c_i$. In the standard case ($L_2$ loss function), $F(p)$ is the sum of squared residuals (and $c_i = 1$).

Every iteration of the algorithm, the residuals and the Jacobian are calculated for a given set of parameters $p$. Then, a new guess $p-\Delta p$ is calculated such that:

$$(J^T W J + \lambda D)\Delta p = J^T W y$$

where $J$ is the Jacobian of the residuals ($J_{ij}=\partial y_i / \partial p_j$), $W$ is a weight matrix depending on the choice of loss function (the identity matrix for $L_2$), $D$ is a diagonal matrix such that $D_{kk} = \max(\epsilon(\lambda), (J^T W J)_{kk})$ with $\epsilon(\lambda)$ the damping floor (dependent on the damping factor), $y$ is the residuals vector, and $\lambda$ is the damping factor. The damping factor determines how much the next guess approximates the prediction of the Gauss-Newton algorithm (lower damping factors) or the gradient descent methods (higher damping factors), in effect defining a trust-region.

For this new guess, the predicted error reduction is calculated as:

$$\Delta s_p = \frac{1}{2}(\Delta p^T J^T W y + \lambda \Delta p^T D \Delta p)$$

If the ratio of the actual error reduction calculated during the next iteration with respect to this prediction is above a defined limit, the current guess is accepted and the damping factor is decreased. Else, the new guess is rejected and the damping factor is increased.

The algorithm finishes once one of these conditions is met, returning the last accepted guess:

* Maximum number of iterations reached
* Cost (sum of losses) below specified tolerance
* Relative change in solution parameters or cost below specified tolerance
* Stagnation at maximum damping factor

### Choice of loss function

The standard $L_2$ loss function calculates the loss as the square of the residual. Its corresponding weight matrix is the identity. Moreover, if an individual scaling factor is defined for each residual, this loss function can be used for the solution of *weighted least-squares* problems.

Robust loss functions provide mechanisms to mitigate the effect of outliers in fitting data:

* **Huber** Equivalent to $L_2$ for small residuals and linear for larger residuals
* **Cauchy** Strongly down-weights large outliers
* **L1-Soft** Smooth approximation of $L_1$ loss that behaves like $L_2$ for small residuals
* **Tukey** Redescending M-estimator that completely rejects extreme outliers (but it may lead to convergence issues if scaling is not chosen well)
* **Welsh** Another redescending M-estimator, smoother than Tukey's in its rejection
* **Fair** Less sensitive to large errors than $L_2$, but not redescending
* **Arctan** Limits maximum loss of single residuals

### Normalized damping factor

A particularity of this LMA implementation is the use of a *normalized damping factor*. The damping factor $\lambda$ will oscillate between the specified limits $\lambda_{min}$ and $\lambda_{max}$, starting with an initial value $\lambda_0$. The normalized damping factor is calculated as:

$$\lambda_{norm} = \frac{(\lambda_{max}-\lambda_0)(\lambda-\lambda_{min})}{(\lambda_0-\lambda_{min})(\lambda_{max}-\lambda)}$$

If $\lambda_{norm}$ is 1, the initial damping factor is being used. If it is larger, it means that more damping than indicated was necessary, reaching the maximum $\lambda_{max}$ at infinite; if it is lower, it means that it could be decreased for faster convergence, with the point at 0 corresponding with $\lambda_{min}$.

The usage of normalized damping factors allows to monitor and adjust the effective size of the trust region during successive function calls independently of the actual damping parameters at use.

### Adaptive damping floor

To enhance numerical stability, particularly when dealing with Jacobian matrices $J$ where $J^T W J$ may have very small or zero diagonal elements, this LMA implementation employs an adaptive floor for the diagonal elements of the damping scaling matrix $D$. The floor $\epsilon(\lambda)$ is calculated such that it takes the value $\epsilon_0$ (an arbitrarily small number) when $\lambda$ is $\lambda_0$ or lower than $\lambda_0$ (so, $\lambda_{norm}≤1$), and it approaches 1 as $\lambda$ approaches $\lambda_{max}$.

This adaptive mechanism ensures that while a very low floor is used during optimistic (low damping) phases, a more substantial floor (approaching 1) is automatically applied to weak components when high overall damping is required.

## Usage

### `Jacobian` Operator

    R←{X}f Jacobian Y
    
`Jacobian` is a monadic operator that takes a monadic function `f` as left operand to return an ambivalent function. This derived function returns an estimation of the Jacobian matrix of `f`, using the method of finite differences. The right argument `Y` is the value at which the Jacobian is calculated, and the optional left argument `X` is the relative perturbation to apply to `Y` in the finite differences method. If `X` is not a given, `⎕CT*÷2` is used.

### `LMA` Operator

    R←{X}f LMA Y

`LMA` is a monadic operator, which takes a left operand to return a derived ambivalent function. This derived function allows to minimize a residual function with a known Jacobian using the Levenberg-Marquardt algorithm, given an initial set of parameters. Several configuration options are available, with sensible defaults previously defined.

The left operand `f` must be a configuration namespace or a function. Configuration namespaces may define the following configuration options:

* `toli`: Maximum number of iterations (default `1E3`)
* `tolc`: Tolerance for the cost (sum of squared residuals or loss values) (default `⎕CT`)
* `tolr`: Tolerance for relative change, either in the solution or the residual (default `⎕CT`)
* `tolg`: Tolerance for the gain ratio to accept or reject a step (default `1E¯2`)
* `dini`: Initial damping factor for `dnorm=1` (default `1E¯2`)
* `dinc`: Increment of damping factor after rejected solution (default `5`)
* `ddec`: Decrement of damping factor after accepted solution (default `÷dinc`)
* `dmax`: Maximum damping factor (default `÷⎕CT`)
* `dmin`: Minimum damping factor (default `÷dmax`)
* `pert`: Relative perturbation applied to parameters for numerical estimation of the Jacobian (default `⎕CT*÷2`)
* `loss`: Choice of loss function: `L2` `Huber` `Cauchy` `L1Soft` `Tukey` `Welsh` `Fair` `Arctan` or dyadic function (default `L2`)
* `scale`: Scale factor passed as left argument to loss function (default for 95% efficiency in robust loss functions)
* `verbose`: If `1`, print `iter cost rel dnorm p` each iteration (default `0`)

Configuration namespaces may also contain the functions:

* `Callback`: Callback function (default `⊢`)
* `Eval`: Evaluation function

The evaluation function `Eval` must return either the residuals and the Jacobian for the given set of solution parameters, or only the residuals. Whenever the residual and Jacobian need to be evaluated, the function `Eval` will be called with trial parameters as right argument and left argument `X`, if given (`Eval` will be called monadically if the derived function `f LMA` is called monadically). `Eval` must return either a two elements vector with the residuals in the first element and the Jacobian in the second one, or a vector of residuals, enclosed if they are not simple scalars. If a Jacobian is not returned, a numerical estimation is calculated evaluating the residual function after applying small perturbations to the parameters (as defined by `pert`).

The function selected by the option `loss` is used to calculate the loss from the residuals and scaling factor. If a function is provided by the user, it must be a dyadic function which returns the loss values and weights when given the residuals as right argument and scaling factor as left argument.

The `Callback` function will be called every iteration before checking convergence, with the current solution namespace as right argument and `X` as left argument, if given (`Callback` will be called monadically if the derived function `f LMA` is called monadically). Its return value is discarded.

If `f` is a function, the result is equivalent to using as `f` a namespace with an `Eval` function `f`
(with default values for the rest of parameters).

`Y` must be a vector.
The first element of `Y`, or `⊂Y` if `1=≡Y`, contains the initial guess for the solution parameters.
If the next element of `Y` is a scalar numeric value, it is interpreted as the initial normalized damping factor.
Additional elements of `Y` must be configuration namespaces. The final configuration parameters are obtained
overwriting the parameters in the namespace given as left operand with those given as right argument from right to left. Default values will be used for non-defined parameters and the `Callback` function, but the `Eval` function must be defined by the user either as left operand `f` or as member of a configuration namespace.

The returned value `R` is a solution namespace corresponding to the last accepted solution.
A solution namespace is a configuration namespace including all the configuration options used to run the algorithm and the additional elements:

* `iter`: Number of iterations
* `cost`: Sum of loss values (squared residuals for L2)
* `rel`: Relative change metric
* `dnorm`: Normalized damping factor
* `p0`: Initial guess
* `p`: Accepted guess

#### Notes

* With the exception of `toli` and `tolc`, configuration parameters should be modified only by expert users or in case of convergence problems

* The relative change metric `rel` is the minimum relative change between successive accepted solutions either in the the cost or in the solution parameters

* In addition to being used for the definition of default values, `⎕CT` is also the baseline for adaptive floor damping

* The perturbation to estimate the Jacobian `pert` and the scaling factor for loss functions `scale` can be either scalar values, or vectors of the same length of respectively the parameters and the residuals

* Loss functions and their respective weights, as well as their default values for the scaling parameter are defined in the namespace `Loss`


### `LM` Operator

`LM` is a simplified version of `LMA`. It is a dyadic operator from which a dyadic function is derived. Usage:

    R←X f LM g Y

where `f` is a monadic evaluation function, `g` is a monadic function which takes as argument an `iter cost rel dnorm p` vector and gets called before every convergence check, `Y` is a two elements vector with the initial guess of parameters and normalized damping factor, and `X` is a vector with the configuration parameters `toli tolc tolr tolg dini dinc ddec dmax dmin`. The function `f` must return either the residuals (as a simple or nested vector), the residuals and the Jacobian, or the residuals, Jacobian, loss values and weights, for a set of parameters. If no Jacobian is provided, it is estimated numerically using `Jacobian` (with no left argument). If no loss values and weights are provided, squared residuals are used.

The return value `R` is an `iter cost rel dnorm p` vector.

## Implementation

In [1]:
)clear

clear ws

In [2]:
]dinput
Jacobian←{⍺←⎕CT*÷2 ⋄ a←⍺×1@(0∘=)|⍵                     ⍝ Jacobian matrix of ⍺⍺ at ⍵ applying perturbation ⍺
    ⍉↑⍺÷⍨(⍺⍺¨(⊂⍵)+↓↑(-⍳≢⍵)↑¨⍺)-⊆⍺⍺ ⍵                   ⍝     finite-differences method
}

In [3]:
]dinput
LM←{ti tc tr tg d0 di dd dx dn←⍺ ⋄ p d←⍵               ⍝ Levenberg–Marquardt algorithm

    D←((dx-d0)×dn-⍨⊢)÷(d0-dn)×dx-⊢ ⋄ L←{⍵,(×⍨⊃⍵)2÷2}   ⍝ damping factor normalization(λ) and standard loss(y)
    J←{(1<|≡g)∧1<≢g←⍺⍺ ⍵:g ⋄ (⊃⊆g)(⍺⍺ Jacobian ⍵)}     ⍝ residual and (estimated if not given) jacobian(p)
    E←⍺⍺{y j l w←L⍣(2=≢g)⊢g←⍺⍺ J ⍵ ⋄ (+/l)⍵ y j w}     ⍝ eval(p): cost, parameters, residual, jacobian
    A←{c p y j w←⍵ ⋄ c p(t+.×j)(y+.×⍨t←w×⍤1⍉j)}        ⍝ accept(EG output): sum(error²), parameters, JtJ, Jty
    
    T←{                                                ⍝ try guess(λ)
        r⊢←0 ⋄ 11::dx⌊⍵×di ⋄ b←1-÷1⌈d⊢←D ⍵             ⍝     bad guess if domain error
        ∆p←jy⌹jj+⍺×⍤1⊢⍵×dj←(⎕CT+b-⎕CT×b)⌈1 1⍉jj        ⍝     change of parameters with adaptive floor
        c0←2÷⍨∆p+.×jy+∆p×⍵×dj                          ⍝     predicted error decrement
        c1←c-⊃g←E⊢q←p-∆p                               ⍝     actual error decrement
        r⊢←(p(-÷⍥(+.×⍨)⊣)q)⌊|c1÷c                      ⍝     relative change in parameters or residuals
        (⎕CT>c0)∧⎕CT<c1:dx⌊⍵×di                        ⍝     if no changing, increase damping
        (⎕CT≤c0)∧tg≥c1÷c0:dx⌊⍵×di                      ⍝     if diverging, increase damping
        dn⌈⍵×dd⊣c p jj jy⊢←A g                         ⍝     accept change, decrease damping
    }
    C←⍵⍵{                                              ⍝ convergence check(λ_prev, λ)
        _←⍺⍺(i⊢←i+1)c r d p                            ⍝     call user function
        (ti<i)∨(dx∧.=⍺ ⍵)∨(tc>c)∨(r>0)∧tr>r            ⍝     iterations, max damping, residual, not changing
    }
    i r←0 ⋄ 0≥ti⊣⍵⍵ g←i c r d p⊣c p jj jy←A E p:g      ⍝ init
    i c r d p⊣(∘.=⍨⍳≢p)T⍣C{11::dx ⋄ D⍣¯1⊢⍵}d           ⍝ iterations, cost, change, norm damping, parameters
}

In [4]:
:Namespace Loss
    l2←1                                               ⍝ standard squared-residual
    L2←{(0.5×⍺××⍨⍵)⍺}
    
    huber←1.345                                        ⍝ ~95% efficiency for normal errors
    Huber←{y←⍺>|⍵ ⋄ Y←y⍨ ⋄ N←~Y                        ⍝ L2 for small residuals, lineal for large ones
        (((⍺÷2)-⍨⍺×|)@N×⍨@Y ⍵)((⍺÷⍨|)@N 1@Y ⍵)
    }
    cauchy←2.385                                       ⍝ ~95% efficiency for normal errors
    Cauchy←{(⍺×(⍺÷2)×⍟1+r)(÷1+r←×⍨⍵÷⍺)}                ⍝ strongly downweights large outliers
    
    softl1←1
    SoftL1←{(⍺×⍺×r-1)(÷r←(1+×⍨⍵÷⍺)*÷2)}                ⍝ L2 for small residuals, L1 for large ones
    
    tukey←4.685                                        ⍝ ~95% efficiency for normal errors
    Tukey←{k←(×⍨⍺)÷6 ⋄ y←⍺>|⍵ ⋄ Y←y⍨ ⋄ N←~Y            ⍝ re-descending M-estimator
        (k@Y(k×1-3*⍨1-⊢)@N⊢r)(0@Y(×⍨1-⊢)@N⊢r←×⍨⍵÷⍺)
    }
    welsh←2.985                                        ⍝ ~95% efficiency for normal errors
    Welsh←{(⍺×(⍺÷2)×1-e)(e←*-×⍨⍵÷⍺)}                   ⍝ smoother re-descending M-estimator
    
    fair←1
    Fair←{(⍺×⍺×r-⍟1+r)(÷1+r←(|⍵)÷⍺)}                   ⍝ no re-descending
    
    arctan←1
    Arctan←{(⍺×(⍺÷2)×¯3○r)(÷1+×⍨r←×⍨⍵÷⍺)}              ⍝ limits maximum loss
:EndNamespace

In [5]:
]dinput
LMA←{⍺←⊢ ⋄ p←⊃w←⊆⍵ ⋄ 0::⎕SIGNAL ⎕EN                    ⍝ pass signals

    n←'d',¨'ini' 'inc' 'dec' 'max' 'min'               ⍝ damping
    n←('icrg',⍨¨⊂'tol'),n                              ⍝ tolerances
    nc←'ddec' 'dmin' ⋄ ns←'scale'                      ⍝ computed defaults
    nr←'iter' 'cost' 'rel' 'dnorm' 'p'                 ⍝ results
    c←(                                                ⍝ default config
        toli:1000 ⋄ tolc:⎕CT ⋄ tolr:⎕CT ⋄ tolg:0.01    ⍝     tolerances: iterations cost relative gain
        dini:0.01 ⋄ dinc:5 ⋄ dmax:÷⎕CT ⋄ loss:'L2'     ⍝     damping and loss function
        p0:p ⋄ pert:⎕CT*÷2 ⋄ verbose:0                 ⍝     init guess, perturbation, logging
    )
    F←{1((⊂6 0⍕↑),12 ¯5∘⍕¨⍤↓)⍵} ⋄ P←{⎕←F ⍵ ⋄ ⍵}        ⍝ format and print
    D←{⍕'sp',¨':',¨(⊂2 5)⌷F ⍵}                         ⍝ display form
    J←{⍉↑⍺÷⍨(⍺⍺¨(⊂⍵)+↓↑(-⍳≢⍵)↑¨⍺)-⊆⍺⍺ ⍵}               ⍝ estimate Jacobian
    E←{⍺←⊢ ⋄ (1<≡e)∧2=≢e←⍺⍺ ⍵:e ⋄ (⊃⊆e)(⍺ ⍺⍺ J ⍵)}     ⍝ evaluate, and estimate J if needed
    M←(2÷⍨1⊥⊢⌷⍨∘⊂⍋⌷⍨∘⊂∘⌈2÷⍨0 1+≢){1@(0∘=)⍺⍺|⍵-⍺⍺ ⍵}    ⍝ median absolute deviation
    L←{0=c.⎕NC'sigma':∇⍵⊣c.sigma←(M ⍵)÷0.6745          ⍝ loss function (needs stddev estimation)
        3=⎕NC'⍺⍺':c.(sigma×⎕VGET⊂ns 1)⍺⍺ ⍵             ⍝     user defined
        ~(⊂⍺⍺)∊Loss.⎕NL¯3:⎕SIGNAL 6                    ⍝     if no user defined it must be in Loss
        c.(sigma×⍣(⍺⍺≢'L2')⊢scale)(Loss.⍎⍺⍺)⍵          ⍝     scaled loss function
    }
    3=⎕NC'⍺⍺':⍺((⍺⍺{⍵.Eval←⍺⍺ ⋄ ⍵}c)∇∇)w               ⍝ ⍺⍺ is Eval function
    (1<≢w)∧~2|⎕DR⊃⌽w:⍺((⎕NS ⍺⍺(⊃⌽w))∇∇)¯1↓w            ⍝ non numeric extra argument is config
    2<≢w:⎕SIGNAL 11                                    ⍝ wrong argument
    
    c.CallBack←⊢ ⋄ c←c ⎕NS ⍺⍺ ⋄ c.dnorm←1⊣⍣(1=≢w)⊃⌽w   ⍝ default callback and actual config
    c.scale←c ⎕VGET⊂ns(Loss ⎕VGET ⎕C c.loss)           ⍝ set scale factor for loss function
    _←c ⎕VSET(↑nc)(c ⎕VGET(↑nc)c.(÷dinc dmax))         ⍝ other computed defaults
    CB←⍺{⍺⍺ c.CallBack c ⎕VSET(↑nr)⍵}P⍣(c.verbose≢0)   ⍝ callback function
    EV←⍺∘c.Eval{y j←⍺ ⍺⍺ E ⍵ ⋄ y j,(c.loss L)y}        ⍝ eval function
    c⊣c.⎕DF D(c ⎕VGET↑n)(c.pert∘EV)LM CB p c.dnorm     ⍝ return namespace
}

### Tests

In [6]:
]dinput
Test←{0::⎕EM ⎕SIGNAL ⎕EN
    t←()
    ⍝ Tests ability to navigate sharp, narrow valleys and handle situations where the Jacobian
    ⍝ may lead to a locally rank-deficient J^T J matrix (eg. zero diagonal elements), requiring
    ⍝ robust damping
    t.Beale←{
        B←{
            x y←⍵ ⋄ (1.5 2.25 2.625-x×1-y*⍳3)[(¯1+y)x ⋄ (¯1+y*2)(2×x×y) ⋄ (¯1+y*3)(3×x×y*2)]
        }
        r←3 0.5 CMP(R B #.LMA 1 0.8).p                                ⍝ easy
        r,←3 0.5 CMP(R B #.LMA 1 1).p                                 ⍝ singular Jt J
        r,←3 0.5 CMP(R B #.LMA 0 0).p                                 ⍝ another tricky point
        r,←3 0.5 CMP(R B #.LMA 1 ¯2).p                                ⍝ different quadrant
        r,←(B #.LMA(2 2)(toli:1)).cost>(R B #.LMA 2 2).cost           ⍝ closer to another local minimum
        r,←(B #.LMA(¯1 1)(toli:1)).cost>(R B #.LMA ¯1 1).cost         ⍝ closer to another local minimum
        r
    }
    ⍝ numerical approximation of the Jacobian
    t.BealeNum←{
        B←{
            x y←⍵ ⋄ (1.5 2.25 2.625-x×1-y*⍳3)
        }
        r←3 0.5 CMP(R B #.LMA 1 0.8).p                                ⍝ easy
        r,←3 0.5 CMP(R B #.LMA 1 1).p                                 ⍝ singular Jt J
        r,←3 0.5 CMP(R B #.LMA 0 0).p                                 ⍝ another tricky point
        r,←3 0.5 CMP(R B #.LMA 1 ¯2).p                                ⍝ different quadrant
        r,←(B #.LMA(2 2)(toli:1)).cost>(R B #.LMA 2 2).cost           ⍝ closer to another local minimum
        r,←(B #.LMA(¯1 1)(toli:1)).cost>(R B #.LMA ¯1 1).cost         ⍝ closer to another local minimum
        r
    }
    ⍝ Jacobian and fitting problem with outliers to test loss functions and bare LM
    ExpDec←{A k C←⍺ ⋄ C+A×*-k×⍵}
    ExpDecEv←{x y←⍺ ⋄ A k C←⍵ ⋄ xe←A×x×e←*-k×x ⋄ (y-⍵ ExpDec x)(⍉(-e)⍪xe⍪⍉⍪-=⍨e)}
    y←100×@(?≢x)⊢0.1{⍵+⍺×0.5-⍨?0⍴⍨≢⍵}y0←(p←10 0.5 1)ExpDec⊢x←(⍳100)-1
    t.ExpDecFit←{p x y y0←#.(p x y y0) ⋄ CMP←{0.1>|⍺-⍵}
        r←p CMP(R x y0∘#.ExpDecEv #.LMA(10 0.5 1)(loss:'L2')).p                    ⍝ exact
        r,←p CMP(R x y0∘#.ExpDecEv #.LMA(10 0.5 1)(loss:'L2' ⋄ scale:?(≢x)⍴0)).p   ⍝ exact (wls)
        r,←p CMP(R x y0∘#.ExpDecEv #.LMA(10 0.5 1)(loss:'Huber')).p                ⍝ exact
        r,←p CMP(R x y∘#.ExpDecEv #.LMA(5 0.1 0.5)(loss:'Cauchy')).p
        r,←p CMP(R x y∘#.ExpDecEv #.LMA(5 0.1 0.5)(loss:'SoftL1')).p
        r,←p CMP(R x y0∘#.ExpDecEv #.LMA(5 0.1 0.5)(loss:'Tukey' ⋄ scale:0.1)).p   ⍝ exact with scaling
        r,←p CMP(R x y∘#.ExpDecEv #.LMA(5 0.1 0.5)(loss:'Welsh')).p
        r,←p CMP(R x y∘#.ExpDecEv #.LMA(5 0.1 0.5)(loss:'Fair')).p
        r,←p CMP(R x y∘#.ExpDecEv #.LMA(5 0.1 0.5)(loss:'Arctan')).p
        r
    }
    t.ExpDecJac←{p x y←#.(p x y0) ⋄ tol←1e¯10
        R←{⎕←(16↑''),12 ¯5⍕⍵ ⋄ tol>⍵}
        r←R+.×⍨,((y-#.ExpDec∘x)#.Jacobian p)-⊃⌽x y #.ExpDecEv p
        r,←R+.×⍨,(1e¯9(y-#.ExpDec∘x)#.Jacobian p)-⊃⌽x y #.ExpDecEv p
        r
    }
    t.ExpDecLM←{p x y←#.(p x y0)
        R←{⎕←(10↑''),((6 0⍕⊃),(12 ¯5⍕2∘⊃),4∘↓)⍵ ⋄ ⍵}
        cfg←1e6 ⎕CT ⎕CT 1e¯2 1e¯2 5(÷5)(÷⎕CT)⎕CT                         ⍝ ti tc tr tg d0 di dd dx dn
        r←p CMP⊃⌽R cfg(y-#.ExpDec∘x)#.LM⊢(5 0.1 0.5)1                    ⍝ only residuals
        r,←p CMP⊃⌽R cfg(x y∘#.ExpDecEv)#.LM⊢(5 0.1 0.5)1                 ⍝ residual and jacobian
        r,←p CMP⊃⌽R cfg((1,⍨⊢,(×⍨1∘↑))x y∘#.ExpDecEv)#.LM⊢(5 0.1 0.5)1   ⍝ everything
        r
    }
    ⍝ Assesses capability to follow a long, narrow, curving multi-dimensional valley, testing
    ⍝ the interplay between step length and direction adjustments controlled by the damping factor
    t.Helical←{
        T←{1|1+(12○⍺+0j1×⍵)÷○2} ⋄ M←{|⍺+0j1×⍵}
        H←{
            a b c←⍵ ⋄ m←a M b ⋄ y←(10×c-10×a T b)(10×m-1)c
            y[((50×b)÷○+.×⍨a b)(-(50×a)÷○+.×⍨a b)10 ⋄ (10×a÷m)(10×b÷m)0 ⋄ 0 0 1]
        }
        r←1 0 0 CMP(R H #.LMA¯1 0 0).p                                ⍝ standard starting point
        r,←1 0 0 CMP(R H #.LMA¯1.2 0.1 0.1).p                         ⍝ slightly perturbed
        r,←1 0 0 CMP(R H #.LMA¯0.9 ¯0.05 ¯0.05).p                     ⍝ in other direction
        r,←1 0 0 CMP(R H #.LMA 0.5 ¯0.5 0.5).p                        ⍝ qualitatively different
        r,←1 0 0 CMP(R H #.LMA¯0.5 0.5 ¯0.5).p                        ⍝ another one
        r,←1 0 0 CMP(R H #.LMA¯1 0 10).p                              ⍝ far off in 3rd dimension
        r,←1 0 0 CMP(R H #.LMA¯1 0 ¯10).p                             ⍝ far off in 3rd dimension
        r,←1 0 0 CMP(R H #.LMA 3 4 5).p                               ⍝ away in all components
        r
    }
    ⍝ Verifies efficiency and correctness for a linear system. Converge should be very fast
    ⍝ (1 or 2 iterations) with minimal damping, demonstrating Gauss-Newton like behavior
    t.Linear←{
        y←(A←?100 10⍴0)+.×x←⍳10 ⋄ s←R{(y-⍨A+.×⍵)A}#.LMA(?10⍴0)0 ⋄ (1=s.iter),(⍳10)CMP s.p
    }
    ⍝ numerical approximation of the Jacobian
    t.LinearNum←{
        y←(A←?100 5⍴0)+.×x←⍳5 ⋄ s←R{y-⍨A+.×⍵}#.LMA(?5⍴0)0 ⋄ (⍳5)CMP s.p
    }
    ⍝ Evaluates LMA's robustness and ability to converge to a solution when the Jacobian
    ⍝ becomes singular (rank-deficient) at or near the optimum
    t.Powell←{
        P←{
            a b c d←⍵ ⋄ r5 r10←5 10*÷2 ⋄ y←(a+10×b)(r5×c-d)(×⍨b-2×c)(r10××⍨a-d)
            y[1 10 0 0 ⋄ 0 0 r5(-r5) ⋄ 0(2×b-2×c)(-2×b-2×c)0 ⋄ (2×r10×a-d)0 0(-2×r10×a-d)]
        }
        r←(4⍴0)CMP(R P #.LMA(3 ¯1 0 1)(tolc:1e¯30)).p                 ⍝ standard starting point
        r,←(4⍴0)CMP(R P #.LMA(0 0 0 0)(tolc:1e¯30)).p                 ⍝ solution (convergence at zero step)
        r,←(4⍴0)CMP(R P #.LMA(1 1 1 1)(tolc:1e¯30)).p                 ⍝ far from solution
        r
    }
    ⍝ Tests LMA's performance in minimizing a classic non-linear function characterized by
    ⍝ a deep, narrow, banana-shaped valley, requiring effective adaptation of search
    ⍝ direction and step size
    Rosenbrock←{p q←⍵ ⋄ ((10×q-×⍨p),1-p)[(-20×p)10 ⋄ ¯1 0]}
    t.Rosenbrock←{
        r←1 1 CMP(R #.Rosenbrock #.LMA 1.5 1.5).p                     ⍝ close to the solution
        r,←1 1 CMP(R #.Rosenbrock #.LMA 2 1).p                        ⍝ not so close
        r,←1 1 CMP(R #.Rosenbrock #.LMA 0 0).p                        ⍝ outside of parabollic valley
        r,←1 1 CMP(R #.Rosenbrock #.LMA ¯1.2 1).p                     ⍝ further
        r,←1 1 CMP(R #.Rosenbrock #.LMA ¯2 ¯2).p                      ⍝ far and wrongly pointed gradient
        r,←1 1 CMP(R #.Rosenbrock #.LMA 2 2).p                        ⍝ far and wrongly pointed gradient
        r
    }
    ⍝ numerical approximation of the Jacobian
    t.RosenbrockNum←{
        r←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 1.5 1.5).p                    ⍝ close to the solution
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 2 1).p                       ⍝ not so close
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 0 0).p                       ⍝ outside of parabollic valley
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA ¯1.2 1).p                    ⍝ further
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA ¯2 ¯2).p                     ⍝ far and wrongly pointed gradient
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 2 2).p                       ⍝ far and wrongly pointed gradient
        r
    }
    ⍝ Check that algorithm finishes on termination conditions
    t.Terminate←{
        r←(R #.Rosenbrock #.LMA(0 0)(toli:10)).(iter>toli)                      ⍝ number of iterations
        r,←(R #.Rosenbrock #.LMA(0 0)(tolc:0)).(rel<tolr)                       ⍝ relative change
        r,←(R #.Rosenbrock #.LMA(0 0)(tolc:0 ⋄ dmax:1e6)).((dnorm>1e6)∧rel=0)   ⍝ maximum damping
        r
    }
    ⍺←1e¯6 ⋄ tol←⍺ ⋄ CMP←{tol>|⍺-⍵} ⋄ R←{⎕←(10↑''),⍵.((6 0⍕iter),(12 ¯5⍕cost),p) ⋄ ⍵}
    (t⎕NS'tol' 'CMP' 'R'){⎕←⍵ ⋄ 0∊⍺⍎⍵,'⍬':('TEST FAILED: ',⍵)⎕SIGNAL 8 ⋄ _←0}¨(↓t.⎕NL 3)⊣⍣(⍵≡'*')⊆⍵
}

In [7]:
Test'*'

Beale 
 6 3.4270E¯21 3 0.4999999999
 9 1.4645E¯21 3 0.5
 32 3.1182E¯17 2.99999998 0.4999999949
 8 2.6016E¯19 3 0.5
 846 2.2605E¯1 ¯15369.87905 1.000064478
 844 2.2605E¯1 ¯15369.90862 1.000064477
BealeNum 
 6 3.4131E¯21 3 0.4999999999
 9 1.4624E¯21 3 0.5
 32 3.1173E¯17 2.99999998 0.4999999949
 8 2.5785E¯19 3 0.5
 816 2.2605E¯1 ¯15421.21242 1.000064263
 824 2.2605E¯1 ¯15392.12945 1.000064384
ExpDecFit 
 1 0.0000E0 10 0.5 1
 1 0.0000E0 10 0.5 1
 1 0.0000E0 10 0.5 1
 8 3.6086E¯1 9.963081164 0.5016881764 1.004084754
 9 9.8500E0 9.962350702 0.5019804723 1.005512492
 1001 3.0048E¯4 9.998625667 0.4999288685 1.000004261
 9 8.5776E¯2 9.963068573 0.5016652526 1.00402812
 10 9.7879E0 9.961998523 0.5022511401 1.006456899
 13 5.2351E¯2 9.963090425 0.5017453135 1.004238847
ExpDecJac 
 6.0285E¯12 
 3.1913E¯12 
ExpDecLM 
 5 9.6447E¯16 9.999999984 0.4999999965 1
 5 9.6867E¯16 9.999999984 0.4999999965 1
 5 1.9373E¯15 9.999999984 0.4999999965 1
Helical 
 10 5.9596E¯20 1 2.177303483E¯10 3.448140016E¯10
 10 5.9231E¯20 1 2.170636163E¯10 3.437591012E¯10
 8 2.3348E¯19 1 4.309357625E¯10 6.82517468E¯10
 14 2.3416E¯20 1 1.364817538E¯10 2.161410116E¯10
 12 7.8592E¯21 1 7.90691E¯11 1.252178359E¯10
 24 1.3710E¯20 1 1.044305611E¯10 1.65382662E¯10
 21 4.2751E¯15 1 5.82997047E¯8 9.234613263E¯8
 10 1.8140E¯20 1 1.201260465E¯10 1.902399678E¯10
Linear 
 1 8.7699E¯25 1 2 3 4 5 6 7 8 9 10
LinearNum 
 1 5.6547E¯16 0.9999999986 1.999999996 3.000000002 3.999999994 5.000000008
Powell 
 57 9.9116E¯31 2.686746826E¯8 ¯2.686746826E¯9 1.720850692E¯8 1.720850692E¯8
 1 0.0000E0 0 0 0 0
 55 9.8879E¯31 ¯2.687933344E¯8 2.687933344E¯9 ¯1.719400136E¯8 ¯1.719400136E¯8
Rosenbrock 
 8 3.7366E¯15 1.000000086 1.00000017
 12 3.9098E¯15 1.000000088 1.000000174
 32 2.5636E¯15 0.999999929 0.999999857
 28 2.4274E¯15 0.9999999309 0.9999998609
 18 2.5797E¯15 0.9999999287 0.9999998566
 14 4.0279E¯15 1.000000089 1.000000176
RosenbrockNum
 8 3.7370E¯15 1.000000086 1.00000017
 12 3.9102E¯15 1.000000088 1.000000174
 32 2.5633E¯15 0.999999929 0.999999857
 28 2.4272E¯15 0.9999999309 0.9999998609
 18 2.5795E¯15 0.9999999287 0.9999998566
 14 4.0284E¯15 1.000000089 1.000000176
Terminate 
 11 5.0000E¯1 0 0
 34 7.1244E¯30 1 1
 13 5.0000E¯1 0 0

### Export

In [8]:
] _←link.export -overwrite # APLSource

## Tutorial

This section provides practical examples to guide in the application of the Levenberg-Marquardt algorithm and the `LMA` and `LMA` operators to the solution of optiomization problems. The examples cover scenarios from simple equation solving to robust non-linear least squares fitting.

### 1. Solving a simple linear system

This example demonstrates the most basic use of `LMA` to solve a known linear system $A p = b$
by minimizing $||A p - b||$. The algorithm should behave like Gauss-Newton for linear problems, converging very quickly to the analytical solution $p = A^{-1} b$.

To begin, we define a random matrix `A` and random vector `b`. The solution parameters `p` can then be found using matrix division with `b⌹A`.

In [9]:
⎕RL←1 ⋄ A←?5 5⍴9 ⋄ b←?5⍴0 ⋄ p←b⌹A
'bAp'⍪⍉⍪⍪¨b A p

┌─────────────┬─────────┬──────────────┐
│b │A │p │
├─────────────┼─────────┼──────────────┤
│0.2650362441 │2 4 5 7 9│ 0.04223479208│
│0.5414495595 │3 6 7 8 2│ 0.2006503854 │
│0.01114182742│6 2 4 4 4│¯0.2872291459 │
│0.6332729126 │8 8 5 1 1│ 0.1614620747 │
│0.1881983241 │9 7 7 3 2│¯0.03512485279│
└─────────────┴─────────┴──────────────┘

Next, we define an evaluation function. This function must return the residuals and Jacobian for a set of solution parameters. The residual is $A p - b$, because the algorithm will minimize the sum of squares of the residual. Therefore, the Jacobian is the matrix $A$.

In [10]:
LinEval←{A b←⍺ ⋄ (b-⍨A+.×⍵)A}     ⍝ residual and jacobian for solution parameters ⍵
'b-A+.×p' 'A'⍪⍉⍪⍪¨A b LinEval p   ⍝ eg: at the solution point the residual should be zero

┌───────────────┬─────────┐
│b-A+.×p │A │
├───────────────┼─────────┤
│1.110223025E¯16│2 4 5 7 9│
│1.110223025E¯16│3 6 7 8 2│
│1.110223025E¯16│6 2 4 4 4│
│1.110223025E¯16│8 8 5 1 1│
│2.220446049E¯16│9 7 7 3 2│
└───────────────┴─────────┘

To specify configuration options, we define a configuration namespace `cfg`. We must provide an evaluation function (the `LinEval` function that we just defined), and we will also set a very tight cost tolerance , since we know that an exact solution exists.

In [11]:
cfg←(tolc:1e¯20)       ⍝ cost tolerance
cfg.Eval←A b∘LinEval   ⍝ evaluation function

Everything else we need to run the algorithm is an initial guess. We use a vector of ones.

In [12]:
r←cfg LMA 5⍴1                    ⍝ run algorithm
'p' 'r.p' '∆'⍪⍉⍪⍪¨p r.p(p-r.p)   ⍝ compare result and known solution

┌──────────────┬──────────────┬────────────────┐
│p │r.p │∆ │
├──────────────┼──────────────┼────────────────┤
│ 0.04223479208│ 0.04223479208│ 2.274291866E¯13│
│ 0.2006503854 │ 0.2006503854 │ 2.296773882E¯13│
│¯0.2872291459 │¯0.2872291459 │¯7.408518243E¯13│
│ 0.1614620747 │ 0.1614620747 │ 4.187206137E¯13│
│¯0.03512485279│¯0.03512485279│¯7.389922008E¯14│
└──────────────┴──────────────┴────────────────┘

The parameters found are very close to the true parameters! We can also inspect other output variables. The sum of squared errors (the cost value) should be low, and we should have achieved convergence very quickly.

In [13]:
RES←{⍺←'iter' 'cost' 'rel' 'dnorm' ⋄ ⍕⍉↑⍺(⍵⎕VGET↑⍺)} ⋄ P←{⍕(¯5↑'p')⍵.p}
RES r ⋄ P r

iter 7 
 cost 1.673636762E¯25
 rel 8.389067503E¯17
 dnorm 6.399999900E¯5 
 p 0.04223479208 0.2006503854 ¯0.2872291459 0.1614620747 ¯0.03512485279

We needed a few iterations to solve the problem. Let's use the `verbose` option to see the progress of the algorithm during each iteration. To pass an additional option without modifying our `cfg` namespace, we will use a namespace in the right argument.

In [14]:
cfg LMA(5⍴1)(verbose:1)

0 1.5181E3 0.0000E0 1.0000E0 1.0000E0 1.0000E0 1.0000E0 1.0000E0 1.0000E0 
 1 2.3662E¯2 9.7451E¯1 1.0000E0 2.3671E¯3 1.2489E¯1 ¯9.4886E¯2 6.3142E¯2 ¯1.7331E¯2 
 2 2.1268E¯3 5.9610E¯1 2.0000E¯1 1.7103E¯2 1.7408E¯1 ¯2.0371E¯1 1.1462E¯1 ¯2.6999E¯2 
 3 4.0359E¯5 9.1412E¯2 4.0000E¯2 3.8707E¯2 1.9708E¯1 ¯2.7573E¯1 1.5496E¯1 ¯3.3979E¯2 
 4 3.8775E¯8 1.3337E¯3 8.0000E¯3 4.2125E¯2 2.0054E¯1 ¯2.8687E¯1 1.6126E¯1 ¯3.5089E¯2 
 5 1.5669E¯12 1.2586E¯6 1.6000E¯3 4.2234E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2 
 6 2.5587E¯18 5.1269E¯11 3.2000E¯4 4.2235E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2 
 7 1.6736E¯25 8.3891E¯17 6.4000E¯5 4.2235E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2 
 s: 1.6736E¯25 p: 4.2235E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2

The iteration number is in the first column; cost, relative change and normalized damping factor are in the second, third and fourth columns; and the accepted solution parameters follow.

We observe that the cost is reduced during each iteration without problems. This indicates that we can reduce the damping factor. Actually, we can reduce it to the minimum using `0` as normalized damping factor.

In [15]:
cfg LMA(5⍴1)0(verbose:1)

0 1.5181E3 0.0000E0 0.0000E0 1.0000E0 1.0000E0 1.0000E0 1.0000E0 1.0000E0 
 1 4.3916E¯25 9.9757E¯1 0.0000E0 4.2235E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2 
 s: 4.3916E¯25 p: 4.2235E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2

Convergence is achieved in a single iteration now!

Notice the flexibility of the different ways in which options can be passed to `LMA`. Alternatively, we can also pass the evaluation function directly, avoiding having to predeclare a namespace. Additional options can be specified in the right argument:

In [16]:
A b LinEval LMA(5⍴1)0(tolc:1e¯20 ⋄ verbose:1)

0 1.5181E3 0.0000E0 0.0000E0 1.0000E0 1.0000E0 1.0000E0 1.0000E0 1.0000E0 
 1 4.3916E¯25 9.9757E¯1 0.0000E0 4.2235E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2 
 s: 4.3916E¯25 p: 4.2235E¯2 2.0065E¯1 ¯2.8723E¯1 1.6146E¯1 ¯3.5125E¯2

### 2. Fundamentals of non-linear least squares (NLLS) fitting

In the next example, we will fit the parameters of a non-linear model. We will consider the exponentical decay function:

$$y = C + A e^{-k x}$$

Let's generate some data:

In [17]:
⎕RL←1
ExpDec←{A k C←⍺ ⋄ C+A×*-k×⍵}                ⍝ exponential decay
Noise←{⍵×1+⍺×0.5-?≠⍨⍵}                      ⍝ add random noise of ±⍺ to ⍵
y0←(p←0.1 5 2)ExpDec x←(⍳100)÷10            ⍝ true data (100 data points)
y←5e¯3 Noise y0                             ⍝ add some random noise (±0.5%)
'x' 'y0' 'y' 'y-y0'⍪⍉⍪(10↑⍪)¨x y0 y(y-y0)   ⍝ first 10 data points

┌───┬───────────┬───────────┬───────────────┐
│x │y0 │y │y-y0 │
├───┼───────────┼───────────┼───────────────┤
│0.1│2.060653066│2.065794637│ 0.005141570806│
│0.2│2.036787944│2.039552083│ 0.002764138412│
│0.3│2.022313016│2.024316539│ 0.002003522932│
│0.4│2.013533528│2.009636489│¯0.003897039077│
│0.5│2.0082085 │2.007880145│¯0.000328354821│
│0.6│2.004978707│2.002851317│¯0.00212739019 │
│0.7│2.003019738│2.007653027│ 0.004633288906│
│0.8│2.001831564│1.997651869│¯0.004179695274│
│0.9│2.0011109 │1.996961868│¯0.004149031743│
│1 │2.000673795│1.998811638│¯0.001862156558│
└───┴───────────┴───────────┴───────────────┘

The `ExpDec` function evaluates the exponential decay function in the selected points for the parameters given. We use it to generate true values `y0` using known parameters `p`. Then, using the function `Noise`, we apply a random variation to all the points (of a maximum of 0.5%), to get the more realistic data in `y`. This imperfect data is a better representation of the kind of data usually found in fitting problems.

To fit the parameters using `LMA`, we need an evaluation function. As in previous example, we are going to define a function that returns the residual and the Jacobian. The Jacobian for the exponentical decay function defined above is given by the matrix:

$$J = \begin{bmatrix}
-e^{-kx_1} & A x_1 e^{-kx_1} & -1 \\
-e^{-kx_2} & A x_2 e^{-kx_2} & -1 \\
\vdots & \vdots & \vdots \\
-e^{-kx_m} & A x_m e^{-kx_m} & -1
\end{bmatrix}
$$

Our evaluation function therefore takes the form:

In [18]:
ExpDecEval←{x y←⍺ ⋄ A k C←⍵ ⋄ xe←A×x×e←*-k×x ⋄ (y-⍵ ExpDec x)(⍉(-e)⍪xe⍪⍉⍪-=⍨e)}   ⍝ residual and jacobian

With the `Eval` function defined, we can now run the LMA solver. We will use again a vector of ones as initial guess. Let's first fit the parameters using the raw output of the `ExpDec` function store in `y0`.

In [19]:
RP←{⎕←RES ⍵ ⋄ ⎕←P ⍵}
RP x y0 ExpDecEval LMA 3⍴1

iter 6 
 cost 7.395448698E¯15
 rel 2.423744115E¯7 
 dnorm 3.199999990E¯4 
 p 0.09999997999 4.999990499 2

The algorithm converges in very few iterations, and the solution parameters are very close to the true parameters (`0.1 5 2`) used to generate `y0`. The cost is extremely low, as expected for a fit to clean data.

Now we attempt to find the parameters using the noisy data in `y`:

In [20]:
RP x y ExpDecEval LMA 3⍴1

iter 9 
 cost 4.227554471E¯4 
 rel 1.367768059E¯16
 dnorm 2.559999000E¯6 
 p 0.113926342 5.363226638 1.999684665

When fitting the noisy data `y`, the algorithm still converges. The solution parameters are reasonably close to the true values, but the final cost is higher, reflecting the noise. The algorithm stopped because the relative change (`rel`) fell below `tolr`, but the cost could not be reduced below tolerance. We can try to refine the solution reducing the `tolr` value.

In [21]:
RP x y ExpDecEval LMA(3⍴1)(tolr:1e¯20)

iter 11 
 cost 4.227554471E¯4 
 rel 1.458118613E¯23
 dnorm 1.023990000E¯7 
 p 0.113926342 5.363226637 1.999684665

Even with a much stricter `tolr`, the solution parameters and cost do not change significantly, indicating that we have found a good local minimum for this noisy dataset.

In [22]:
(x y ExpDecEval LMA p(toli:0 ⋄ verbose:1)).cost

0 4.4650E¯4 0.0000E0 1.0000E0 1.0000E¯1 5.0000E0 2.0000E0 
0.0004464956831

By running LMA with the true parameters `p` and `toli:0` (0 iterations, so just one evaluation), the resulting cost shows the inherent sum of losses for a perfect model with noisy data. This is the theoretical minimum cost `LMA` could achieve for this noisy data with these true parameters. The algorithm actually yielded a slightly lower cost, indicating that we are *overfitting* (adjusting the parameters to the noise instead of real data).

#### Numerical estimation of Jacobian matrix

What if we do not know what is the Jacobian and only have a residual function available? In this case, the algorithm can still find a solution, using a numerical estimation of the Jacobian.

Let's consider the exponential decay function again. But, this time, we will not use the `ExpDecEval` function. Instead, we define an `ExpDecRes` function that only returns the residuals:

In [23]:
ExpDecRes←{x y←⍺ ⋄ y-⍵ ExpDec x}
RP x y ExpDecRes LMA(3⍴1)

iter 9 
 cost 4.227554471E¯4 
 rel 2.308147563E¯15
 dnorm 2.559999000E¯6 
 p 0.1139263484 5.363227017 1.999684665

Although it took a few more iterations, the results are very similar to those obtained with the analytical Jacobian. `LMA` automatically detected that `ExpDecRes` only returned residuals and used the `Jacobian` operator (with the default `pert` value `⎕CT*÷2`) to approximate the Jacobian.

Modifying the configuration parameter `pert`, it is possible to adjust the applied perturbation:

In [24]:
RP x y ExpDecRes LMA(3⍴1)(pert:⎕CT*÷2)   ⍝ default perturbation
RP x y ExpDecRes LMA(3⍴1)(pert:1e0)      ⍝ large perturbation, worse prediction makes convergence slower
RP x y ExpDecRes LMA(3⍴1)(pert:1e¯15)    ⍝ perturbation too small, inacurate result

iter 9 
 cost 4.227554471E¯4 
 rel 2.308147563E¯15
 dnorm 2.559999000E¯6 
 p 0.1139263484 5.363227017 1.999684665

iter 23 
 cost 4.227573288E¯4 
 rel 4.524970054E¯15
 dnorm 2.500000000E1 
 p 0.1137824911 5.354739109 1.999683407

iter 31 
 cost 1.710624289E¯3 
 rel 5.578222705E¯16
 dnorm 9.765625010E6 
 p 0.03468243617 1.138692831 1.998389309

Very large perturbations can degrade the Jacobian approximation, resulting in slow convergence, while extremely small ones (like `1E-15`) can lead to numerical inaccuracies and poor solutions due to precision limits in finite differences. The default perturbation provides a good balance in this case.

For an accurate result, the Jacobian calculated numerically should be close to the analytical Jacobian. However, the quality of the approximation can vary depending on the actual solution parameters being used, as we can see comparing the Jacobian calculated numerically with the analytical Jacobian defined earlier. We calculate a relative error for the initial and final solution parameters:

In [25]:
(⊃⌽x y ExpDecEval 3⍴1)(-÷⍥(+.×⍨,)⊣)x y∘ExpDecRes Jacobian 3⍴1  ⍝ initial relative error
(⊃⌽x y ExpDecEval p)(-÷⍥(+.×⍨,)⊣)x y∘ExpDecRes Jacobian p      ⍝ final relative error

1.747428903E¯16

5.794393457E¯18

The relative error between the analytical Jacobian and our numerical approximation (using the default perturbation) is very small, especially for the final solution parameters, confirming the reliability of the numerical estimation.

#### Callback function

We could be interested in monitoring some variable during the optimization process. For example, in the last example, we might want to know how the numerical approximation of the Jacobian is progressing. This can be achieved using the callback function. The `CallBack` function, defined in the configuration namespace, is run every iteration with a namespace argument like the one returned by `LMA` as result, allowing the user to perform a detailed diagnosis.

Here, we define a callback function that displays the relative error with respect to the analytical Jacobian every iteration, together with the cost, relative error, and last accepted guess for the solution parameters.

In [26]:
cfg←(tolr:1e¯6) ⋄ cfg.Eval←ExpDecRes ⋄ cfg.ExpDec←ExpDec
cfg.CallBack←{⎕←12 ¯5⍕⍵.(cost rel,p),⍨(⊃⌽#.(x y ExpDecEval p))(-÷⍥(+.×⍨,)⊣)#.(x 0∘ExpDecRes Jacobian)⍵.p}
⎕←∊⍕10↑¨'errJ' 'cost' 'rel' 'A' 'k' 'C' ⋄ x y(cfg LMA)3⍴1

errJ cost rel A k C 
 5.1178E¯2 4.2757E1 0.0000E0 1.0000E0 1.0000E0 1.0000E0 
 2.6863E¯2 9.5101E¯3 6.1341E¯1 6.6774E¯2 9.9423E¯1 1.9845E0 
 1.0724E¯2 1.5729E¯3 7.7278E¯2 4.9287E¯2 1.6111E0 2.0007E0 
 1.4052E¯3 6.8826E¯4 3.4073E¯1 6.8879E¯2 3.1107E0 2.0002E0 
 4.2633E¯6 4.4088E¯4 2.2285E¯1 9.9420E¯2 4.8565E0 1.9998E0 
 2.4487E¯5 4.2288E¯4 9.6043E¯3 1.1335E¯1 5.3712E0 1.9997E0 
 2.3480E¯5 4.2276E¯4 2.0250E¯6 1.1392E¯1 5.3630E0 1.9997E0 
 2.3505E¯5 4.2276E¯4 1.2926E¯9 1.1393E¯1 5.3632E0 1.9997E0 
 s: 4.2276E¯4 p: 1.1393E¯1 5.3632E0 1.9997E0

### 3. Fitting of a physical model with experimental data

At continuation, we are going to tackle a problem closer to a real life example. We will see how to fit the parameters of a Gaussian function:

$$f(x) = c + a \cdot \exp\left(-\frac{(x-m)^2}{2s^2}\right)$$

Gaussian peaks are arguably one of the most frequently encountered shapes in scientific and engineering data (spectroscopy, chromatography, signal processing, statistical distributions, etc.). Its symmetric, peak-like structure behaves differently from the monotonic exponential decay function in previous example.

For the fitting, we are going to produce virtual experimental data. First, we generate clean output data using a predefined set of parameters. Next, some small noise is added to the data, using the same `Noise` function from previous example. Finally, we create a few outliers applying a much larger perturbation, again using the `Noise` function, but with a larger value as left argument and applying it only to a few random points.

In [27]:
⎕RL←1 ⋄ X←{m s←⍺ ⋄ *-(×⍨⍵-m)÷(2××⍨s)}
Gauss←{a s m c←⍺ ⋄ c+a×m s X ⍵}                                ⍝ gauss function
y0←(p←5 2 1.5 0.1)Gauss x←0.25×(⍳20)-1                         ⍝ true data
y←(rl←1 Noise@(5?≢x)=⍨x)×y1←(rs←5e¯2 Noise=⍨x)×y0              ⍝ random noise (±5%) and 5 outliers (±100%)
'x' 'y0' '∆small' 'y1' '∆large' 'y'⍪⍉⍪⍪¨x y0(1-rs)y1(1-rl)y

┌────┬───────────┬────────────────┬───────────┬──────────────┬───────────┐
│x │y0 │∆small │y1 │∆large │y │
├────┼───────────┼────────────────┼───────────┼──────────────┼───────────┤
│0 │3.87419801 │¯0.0249511715 │3.970863789│ 0 │3.970863789│
│0.25│4.212887812│¯0.01357106625 │4.270061192│¯0.2613420563 │5.386007764│
│0.5 │4.512484513│¯0.009907086175 │4.557190086│ 0 │4.557190086│
│0.75│4.760512462│ 0.01935422987 │4.668376409│¯0.4749492285 │6.885618183│
│1 │4.946166172│ 0.001635063396 │4.938078877│ 0 │4.938078877│
│1.25│5.061089691│ 0.01061053757 │5.007388809│ 0 │5.007388809│
│1.5 │5.1 │¯0.02313151897 │5.217970747│ 0 │5.217970747│
│1.75│5.061089691│ 0.02087935543 │4.955417401│ 0 │4.955417401│
│2 │4.946166172│ 0.02073364222 │4.843614133│ 0 │4.843614133│
│2.25│4.760512462│ 0.009307647067 │4.716203292│ 0 │4.716203292│
│2.5 │4.512484513│¯0.01474302463 │4.579012183│¯0.02582317868│4.697256833│
│2.75│4.212887812│¯0.01809174955 │4.289106323│ 0 │4.289106323│
│3 │3.87419801 │¯0.01595143831 │3.93599704 │ 0 │3.93599704 │
│3.25│3.509703756│ 0.0009355548021│3.506420236│ 0 │3.506420236│
│3.5 │3.132653299│¯0.006445134993 │3.152843672│ 0 │3.152843672│
│3.75│2.755479955│ 0.006737319096 │2.736915407│ 0.2338351955 │2.096928258│
│4 │2.389166809│ 0.007569904963 │2.371081043│ 0 │2.371081043│
│4.25│2.042790638│¯0.01855836257 │2.080701487│ 0 │2.080701487│
│4.5 │1.723262337│¯0.004494514645 │1.731007565│ 0 │1.731007565│
│4.75│1.435259176│ 0.01540599528 │1.41314758 │¯0.1043841607 │1.560657804│
└────┴───────────┴────────────────┴───────────┴──────────────┴───────────┘

We will fit the parameters using a numerical approximation of the Jacobian (we will also use the analytical one later):

In [28]:
GaussRes←{x y←⍺ ⋄ y-⍵ Gauss x}
RP x y1 GaussRes LMA(4⍴1)(verbose:0)
RP x y GaussRes LMA(4⍴1)(verbose:0)

iter 16 
 cost 3.260096722E¯2 
 rel 4.818105743E¯20
 dnorm 1.279999900E¯5 
 p 5.360498897 2.151620676 1.483878994 ¯0.2896122654

iter 15 
 cost 2.235416692E0 
 rel 3.306121866E¯16
 dnorm 2.559999000E¯6 
 p 4.956807586 2.014555465 1.202540192 0.4350967403

The first run on the data with small noise (`y0`) converges well, with the solution parameters reasonably close to the true values `5 2 1.5 0.1`. However, the second run on the data with significant outliers (`y`) shows that the L2 loss function is heavily influenced by these outliers, and the resulting parameters are poor, with a high final cost.

#### Weighted least-squares fitting

If we know beforehand what is the quality of each of our datapoints--for example, because the experimental device that we use give us data about the quality of a measurement--we can use this information to guide the fitting process. Assigning a different weight to each of the data points, we can perform what is called a *weighted least-squares* (WLS) fitting.

In [29]:
RP x y1 GaussRes LMA(4⍴1)(scale:s1←÷1+|y1-y0)
RP x y GaussRes LMA(4⍴1)(scale:s←÷1+|y-y0)

iter 16 
 cost 2.994435871E¯2 
 rel 1.743218715E¯16
 dnorm 1.279999900E¯5 
 p 5.353038299 2.149186423 1.48366159 ¯0.2820439403

iter 15 
 cost 1.006671035E0 
 rel 8.914381500E¯18
 dnorm 2.559999000E¯6 
 p 5.058256596 2.079191318 1.327590531 0.1594953187

By applying weights (here, inversely proportional to the deviation from the true data `y0`), the WLS fit gives more importance to data points believed to be more reliable. For the `y1` data, the parameters are similar to the unweighted L2 fit. However, for the `y` data with outliers, WLS can improve the fit if the weights correctly down-weight the outlier points.

#### Robust loss functions

Data about the quality of a measurement is not always available. It is often the case that experimental data present noise and outliers, but we do not know in advance how to weight our data points to get an accurate result. In this case, the use of *robust loss functions* can help to find a better solution.

We are going to repeat the fitting of the Gaussian function parameters using different robust functions:

In [30]:
⊢r←x y1 GaussRes LMA(4⍴1)
⊢r,←x y1 GaussRes LMA(4⍴1)(scale:s1)        ⍝ wls
⊢r,←x y1 GaussRes LMA(4⍴1)(loss:'Huber')
⊢r,←x y1 GaussRes LMA(4⍴1)(loss:'Cauchy')
⊢r,←x y1 GaussRes LMA(4⍴1)(loss:'SoftL1')
⊢r,←x y1 GaussRes LMA(4⍴1)(loss:'Tukey')
⊢r,←x y1 GaussRes LMA(4⍴1)(loss:'Welsh')
⊢r,←x y1 GaussRes LMA(4⍴1)(loss:'Fair')
⊢r,←x y1 GaussRes LMA(4⍴1)(loss:'Arctan')

s: 3.2601E¯2 p: 5.3605E0 2.1516E0 1.4839E0 ¯2.8961E¯1

s: 2.9944E¯2 p: 5.3530E0 2.1492E0 1.4837E0 ¯2.8204E¯1

s: 6.5202E¯2 p: 5.3605E0 2.1516E0 1.4839E0 ¯2.8961E¯1

s: 3.2551E¯2 p: 5.3609E0 2.1519E0 1.4838E0 ¯2.9016E¯1

s: 3.2461E¯2 p: 5.3617E0 2.1523E0 1.4838E0 ¯2.9113E¯1

s: 4.5500E1 p: 1.0000E0 1.0000E0 1.0000E0 1.0000E0

s: 3.2569E¯2 p: 5.3608E0 2.1518E0 1.4839E0 ¯2.8996E¯1

s: 3.0204E¯2 p: 5.3659E0 2.1555E0 1.4828E0 ¯2.9694E¯1

s: 3.2596E¯2 p: 5.3606E0 ¯2.1517E0 1.4839E0 ¯2.8976E¯1

With the exception of Tukey's loss function, all of them converge with a relatively low cost. Tukey's function usually requires a careful adjustment of the scaling factor.

In [31]:
⊢r[5]←x y1 GaussRes LMA(4⍴1)(loss:'Tukey' ⋄ scale:0.1)

s: 2.3200E¯2 p: 5.0030E0 1.9893E0 1.4905E0 1.0167E¯1

To compare all loss functions, we can calculate the sum of squared residuals corresponding to the solution parameters obtained using each of the loss functions.

In [32]:
rl2←(x y1(GaussRes LMA)(toli:0),⍨⊂)¨r.p
'Robust' 'L2'⍪⍉⍪(12 ¯5⍕⍪)¨r.cost rl2.cost

┌────────────┬────────────┐
│Robust │L2 │
├────────────┼────────────┤
│ 3.2601E¯2 │ 3.2601E¯2 │
│ 2.9944E¯2 │ 3.2604E¯2 │
│ 6.5202E¯2 │ 3.2601E¯2 │
│ 3.2551E¯2 │ 3.2601E¯2 │
│ 2.3200E¯2 │ 4.6172E¯2 │
│ 4.5500E1 │ 6.5772E1 │
│ 3.2569E¯2 │ 3.2601E¯2 │
│ 3.0204E¯2 │ 3.2617E¯2 │
│ 3.2596E¯2 │ 3.2601E¯2 │
└────────────┴────────────┘

The `Robust` column shows the sum of the respective loss values, which are not directly comparable across different loss types. However, in the `L2` column, we see that when fitting the data with only small noise, most robust loss functions yield a final sum of squared residuals very similar to that obtained by the direct L2 fit. This demonstrates that robust loss functions do not significantly penalize performance on clean data.

In [33]:
r←x y GaussRes LMA(4⍴1)
r,←x y GaussRes LMA(4⍴1)(scale:s)         ⍝ wls
r,←x y GaussRes LMA(4⍴1)(loss:'Huber')
r,←x y GaussRes LMA(4⍴1)(loss:'Cauchy')
r,←x y GaussRes LMA(4⍴1)(loss:'SoftL1')
r,←x y GaussRes LMA(4⍴1)(loss:'Tukey')
r,←x y GaussRes LMA(4⍴1)(loss:'Welsh')
r,←x y GaussRes LMA(4⍴1)(loss:'Fair')
r,←x y GaussRes LMA(4⍴1)(loss:'Arctan')
rl2←(x y(GaussRes LMA)(toli:0),⍨⊂)¨r.p
'Robust' 'L2'⍪⍉⍪(12 ¯5⍕⍪)¨r.cost rl2.cost

┌────────────┬────────────┐
│Robust │L2 │
├────────────┼────────────┤
│ 2.2354E0 │ 2.2354E0 │
│ 1.0067E0 │ 2.4495E0 │
│ 3.2862E0 │ 2.2842E0 │
│ 1.7824E0 │ 2.2950E0 │
│ 1.5281E0 │ 2.3450E0 │
│ 3.7777E1 │ 1.2132E1 │
│ 1.8620E0 │ 2.2868E0 │
│ 1.1538E0 │ 2.3600E0 │
│ 9.0551E¯1 │ 2.8876E0 │
└────────────┴────────────┘

When fitting the data containing significant outliers, the benefit of robust loss functions becomes clear. While the L2 fit is heavily skewed (high L2 cost), functions like Huber, Cauchy, SoftL1, Fair, and Arctan achieve a much lower sum of squared residuals by down-weighting the outliers. Their own cost is also minimized effectively. Tukey and Welsh, being redescending, can be very effective but may require careful tuning of their scale parameter, or more iterations, to converge optimally, especially if the initial scale (auto-estimated or default) is not ideal for the outlier distribution.

#### `LM` operator

The `LMA` operator provides a convenient interface for the user. However, the core engine can also be directly accessed using the `LM` operator (which `LMA` uses internally).

Although the interface for `LM` is more "arcane", it offers the same flexibility as `LMA`. The evaluation function must be the left operand. The callback function (which in this case takes a vector instead of namespace) is specified as the right operand. All configuration parameters need to be passed as left argument, and the right argument must be a two elements vector with the initial parameters and normalized damping factor.

The evaluation function may return either only the residual, the residual and the Jacobian, or also loss and weight values if it is desired to use a non-standard loss function. Let's see these three cases for the Gaussian function. We will need the analytical Jacobian given by:

$$
J_i = \begin{bmatrix}
-e^{-\frac{(x_i-m)^2}{2s^2}} & 
-a \cdot e^{-\frac{(x_i-m)^2}{2s^2}} \cdot \frac{(x_i-m)^2}{s^3} & 
-a \cdot e^{-\frac{(x_i-m)^2}{2s^2}} \cdot \frac{(x_i-m)}{s^2} & 
-1
\end{bmatrix}
$$

In [34]:
GaussJac←{a s m c←⍺ ⋄ x2←(x1←(xe←-m s X ⍵)×a×(⍵-m)÷×⍨s)×(⍵-m)÷s ⋄ ⍉↑(xe)x2(x1)(-=⍨xe)}
(p GaussJac x)(-÷⍥(+.×⍨,)⊣)x y∘GaussRes Jacobian p   ⍝ check error with respect to numerical estimation

1.014351005E¯15

We can now define three different evaluation functions (notice that they must be monadic functions, so we bind the left argument with `∘`):

In [35]:
GaussEval1←x y∘GaussRes                         ⍝ residual only
GaussEval2←x y∘{(⍺ GaussRes ⍵)(⍵ GaussJac⊃⍺)}   ⍝ residual and jacobian
GaussEval3←(⊢,Loss.(fair∘Fair)⍤⊃)GaussEval2     ⍝ residual, jacobian, and fair loss function

Finally, we run the algorithm using the default configuration parameters of `LMA`, and a `CallBack` function with a functionality similar to `verbose:1`.

In [36]:
cfg←1e3 ⎕CT ⎕CT 1e¯2 1e¯2 5(÷5)(÷⎕CT)⎕CT   ⍝ ti tc tr tg d0 di dd dx dn
Verbose←{⎕←(3 0⍕⊃⍵),12 9∘⍕¨1↓⍵}
cfg GaussEval1 LM Verbose(4⍴1)1
cfg GaussEval2 LM Verbose(4⍴1)1
cfg GaussEval3 LM Verbose(4⍴1)1

0 77.215082377 0.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
 1 11.987065609 0.844757459 1.000000000 3.162978849 2.727267330 1.272949344 2.030246450
 2 11.987065609 0.228400446 0.200000000 3.162978849 2.727267330 1.272949344 2.030246450
 3 11.987065609 0.208087108 1.000000000 3.162978849 2.727267330 1.272949344 2.030246450
 4 11.987065609 0.151019983 5.000000000 3.162978849 2.727267330 1.272949344 2.030246450
 5 6.247480602 0.088946997 25.000000000 3.239695395 1.410734496 0.712543741 1.935310079
 6 2.863887822 0.026737963 5.000000000 3.550801485 1.782248042 1.097438736 1.681121690
 7 2.292328953 0.046311013 1.000000000 4.309319875 1.760002360 1.236176002 1.113172560
 8 2.239637714 0.020148103 0.200000000 4.768992457 1.959750725 1.202078465 0.623314833
 9 2.235452415 0.001868739 0.040000000 4.943812834 2.011998201 1.201744836 0.448049541
 10 2.235416699 0.000011297 0.008000000 4.956738360 2.014570638 1.202505523 0.435159628
 11 2.235416692 0.000000000 0.001600000 4.956793802 2.014549004 1.202541183 0.435111833
 12 2.235416692 0.000000000 0.000320000 4.956809012 2.014556181 1.202540032 0.435095170
 13 2.235416692 0.000000000 0.000064000 4.956807512 2.014555420 1.202540206 0.435096824
 14 2.235416692 0.000000000 0.000012800 4.956807652 2.014555491 1.202540190 0.435096670
 15 2.235416692 0.000000000 0.000002560 4.956807586 2.014555465 1.202540192 0.435096740
┌──┬───────────┬───────────────┬──────────────┬────────────────────────────────────────────────┐
│15│2.235416692│3.306121866E¯16│0.000002559999│4.956807586 2.014555465 1.202540192 0.4350967403│
└──┴───────────┴───────────────┴──────────────┴────────────────────────────────────────────────┘

0 77.215082377 0.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
 1 11.987064561 0.844757472 1.000000000 3.162978743 2.727267116 1.272949392 2.030246543
 2 11.987064561 0.228400321 0.200000000 3.162978743 2.727267116 1.272949392 2.030246543
 3 11.987064561 0.208087028 1.000000000 3.162978743 2.727267116 1.272949392 2.030246543
 4 11.987064561 0.151019570 5.000000000 3.162978743 2.727267116 1.272949392 2.030246543
 5 6.247478919 0.088946973 25.000000000 3.239695322 1.410734519 0.712543835 1.935310179
 6 2.863887746 0.026737963 5.000000000 3.550801494 1.782247989 1.097438792 1.681121707
 7 2.292328945 0.046311014 1.000000000 4.309319889 1.760002400 1.236175975 1.113172536
 8 2.239637713 0.020148101 0.200000000 4.768992459 1.959750724 1.202078460 0.623314832
 9 2.235452415 0.001868739 0.040000000 4.943812783 2.011998181 1.201744836 0.448049593
 10 2.235416699 0.000011297 0.008000000 4.956738374 2.014570647 1.202505520 0.435159611
 11 2.235416692 0.000000000 0.001600000 4.956793822 2.014549012 1.202541182 0.435111813
 12 2.235416692 0.000000000 0.000320000 4.956809043 2.014556195 1.202540029 0.435095137
 13 2.235416692 0.000000000 0.000064000 4.956807411 2.014555378 1.202540211 0.435096932
 14 2.235416692 0.000000000 0.000012800 4.956807614 2.014555479 1.202540189 0.435096709
┌──┬───────────┬───────────────┬──────────────┬────────────────────────────────────────────────┐
│14│2.235416692│1.191963569E¯15│0.000012799999│4.956807614 2.014555479 1.202540189 0.4350967092│
└──┴───────────┴───────────────┴──────────────┴────────────────────────────────────────────────┘

0 26.896757586 0.000000000 1.000000000 1.000000000 1.000000000 1.000000000 1.000000000
 1 6.334011983 0.764506485 1.000000000 3.145538156 2.936213343 1.416108353 1.886657074
 2 6.334011983 0.291511045 0.200000000 3.145538156 2.936213343 1.416108353 1.886657074
 3 6.334011983 0.282114932 1.000000000 3.145538156 2.936213343 1.416108353 1.886657074
 4 6.334011983 0.239942452 5.000000000 3.145538156 2.936213343 1.416108353 1.886657074
 5 5.060116840 0.141468387 25.000000000 3.212505577 1.203909890 0.783109994 1.862235960
 6 2.035570084 0.023462366 5.000000000 3.271190190 1.754097324 1.024785457 1.776564669
 7 1.446586300 0.081402255 1.000000000 4.188654453 1.870188441 1.297924884 1.045737834
 8 1.329907437 0.036226904 0.200000000 4.847286551 2.038210604 1.279424701 0.412826371
 9 1.322673316 0.004472512 0.040000000 5.097415256 2.116483396 1.267973422 0.161971001
 10 1.322431553 0.000174928 0.008000000 5.149268470 2.133744563 1.263979287 0.110863044
 11 1.322422191 0.000001180 0.001600000 5.153698088 2.135566420 1.262764660 0.107102549
 12 1.322421261 0.000000033 0.000320000 5.154417472 2.135970881 1.262370709 0.106612872
 13 1.322421162 0.000000003 0.000064000 5.154646793 2.136106671 1.262241305 0.106456716
 14 1.322421151 0.000000000 0.000012800 5.154722464 2.136151784 1.262198902 0.106404759
 15 1.322421150 0.000000000 0.000002560 5.154747289 2.136166606 1.262185023 0.106387673
 16 1.322421150 0.000000000 0.000000512 5.154755416 2.136171460 1.262180482 0.106382075
 17 1.322421150 0.000000000 0.000000102 5.154758076 2.136173048 1.262178996 0.106380243
 18 1.322421150 0.000000000 0.000000020 5.154758946 2.136173568 1.262178510 0.106379644
 19 1.322421150 0.000000000 0.000000004 5.154759231 2.136173738 1.262178351 0.106379448
┌──┬──────────┬───────────────┬────────┬────────────────────────────────────────────────┐
│19│1.32242115│5.306258386E¯15│4.095E¯9│5.154759231 2.136173738 1.262178351 0.1063794478│
└──┴──────────┴───────────────┴────────┴────────────────────────────────────────────────┘